# Image retrieval using deep features

In [ ]:
import turicreate
from turicreate import SArray
from turicreate import SFrame

# Load some CIFAR-10 images

In [2]:
image_data = turicreate.SFrame('~/venv/Apple ML foundations/image_train_data/')

In [3]:
image_data['image'].explore()

Materializing SFrame

# Compute deep features from our images

In [ ]:
#deep_learning_model = turicreate.load_model('imagenet_model_iter45')
#image_data['deep_features'] = deep_learning_model.extract_features(image_data)

In [4]:
image_data

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.24287176132202148,1.0954537391662598, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.5250879526138306, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.5660159587860107, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.129795789718628, 0.0,0.0, 0.7781944870948792, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.7178692817687988, 0.0,0.0, 0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.5781855583190918, 0.0,0.0, 0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.22067785263061523, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0,0.23753464221954346, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.57378625869751, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.6589357256889343, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


# Create a nearest neighbors model to retrieve images from deep features

In [5]:
knn_model = turicreate.nearest_neighbors.create(image_data,
                                               features = ['deep_features'],
                                               label = 'id')

Starting brute force nearest neighbors model training.

# Use image retrieval model with deep features to find similar images

In [ ]:
cat = image_data[18:19]

In [ ]:
cat['image'].explore()

In [ ]:
knn_model.query(cat)

In [ ]:
def get_images_from_ids(query_result):
    return image_data.filter_by(query_result['reference_label'],'id')

In [ ]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))

In [ ]:
cat_neighbors['image'].explore()

# Find images similar to a car

In [ ]:
car = image_data[8:9]

In [ ]:
car['image'].explore()

In [ ]:
get_images_from_ids(knn_model.query(car))['image'].explore()

# Create a lambda function to find and show nearest neighbors to an image

In [ ]:
show_neighbors = lambda i: get_images_from_ids(knn_model.query(image_data[i:i+1]))['image'].explore()

In [ ]:
show_neighbors(8)

In [ ]:
show_neighbors(26)

In [ ]:
show_neighbors(500)

## Task 1

In [ ]:
sketch = turicreate.Sketch(image_data['label'])

In [ ]:
sketch

In [ ]:
image_data['label'].summary()

## Task 2

In [6]:
sf = turicreate.SFrame('~/venv/Apple ML foundations/image_train_data/')
image_test = turicreate.SFrame('~/venv/Apple ML foundations/image_test_data/')

In [7]:
dog=sf[sf['label'].apply(lambda x: x== 'dog')]
cat=sf[sf['label'].apply(lambda x: x== 'cat')]
automobile=sf[sf['label'].apply(lambda x: x== 'automobile')]
bird=sf[sf['label'].apply(lambda x: x== 'bird')]

In [8]:
knn_dog = turicreate.nearest_neighbors.create(dog,
                                               features = ['deep_features'],
                                               label = 'id');
knn_cat = turicreate.nearest_neighbors.create(cat,
                                               features = ['deep_features'],
                                               label = 'id');
knn_automobile = turicreate.nearest_neighbors.create(automobile,
                                               features = ['deep_features'],
                                               label = 'id');
knn_bird = turicreate.nearest_neighbors.create(bird,
                                               features = ['deep_features'],
                                               label = 'id');

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

In [9]:
cat1=image_test[0:1]
cat1['image'].explore()

Materializing SFrame

In [15]:
result1=knn_cat.query(cat1)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 5.658ms      |

| Done         |         | 100         | 37.602ms     |

+--------------+---------+-------------+--------------+

In [16]:
result2=knn_dog.query(cat1)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 6.445ms      |

| Done         |         | 100         | 35.581ms     |

+--------------+---------+-------------+--------------+

In [10]:
def get_images_from_ids(query_result):
    return image_data.filter_by(query_result['reference_label'],'id')

In [13]:
get_images_from_ids(knn_cat.query(image_test[0:1]))['image'].explore()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 9.732ms      |

| Done         |         | 100         | 42.462ms     |

+--------------+---------+-------------+--------------+

Materializing SFrame

In [12]:
get_images_from_ids(knn_dog.query(cat1))['image'].explore()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 9.375ms      |

| Done         |         | 100         | 43.134ms     |

+--------------+---------+-------------+--------------+

Materializing SFrame

## Task 3

In [17]:
result1

query_label,reference_label,distance,rank
0,16289,34.62371920804245,1
0,45646,36.00687992842462,2
0,32139,36.52008134363789,3
0,25713,36.754850252057054,4
0,331,36.87312281675268,5


In [18]:
result2

query_label,reference_label,distance,rank
0,16976,37.464262878423774,1
0,13387,37.56668321685285,2
0,35867,37.60472670789396,3
0,44603,37.70655851529755,4
0,6094,38.511325490739715,5


In [19]:
result1['distance'].mean()

36.15573070978294

In [20]:
result2['distance'].mean()

37.77071136184157

## Task 4

In [21]:
image_test = turicreate.SFrame('~/venv/Apple ML foundations/image_test_data/')
image_test_dog=image_test[image_test['label'].apply(lambda x: x== 'dog')]
image_test_cat=image_test[image_test['label'].apply(lambda x: x== 'cat')]
image_test_automobile=image_test[image_test['label'].apply(lambda x: x== 'automobile')]
image_test_bird=image_test[image_test['label'].apply(lambda x: x== 'bird')]

In [22]:
image_test_dog

id,image,label,deep_features,image_array
12,Height: 32 Width: 32,dog,"[0.3223174810409546, 0.0,1.2493335008621216, 0.0, ...","[91.0, 64.0, 30.0, 82.0,58.0, 30.0, 87.0, 73.0, ..."
16,Height: 32 Width: 32,dog,"[0.0, 0.0,0.34735703468322754, ...","[95.0, 76.0, 78.0, 92.0,77.0, 78.0, 89.0, 77.0, ..."
24,Height: 32 Width: 32,dog,"[1.3155765533447266, 0.0,0.0, 0.0, 0.0, 0.0, ...","[136.0, 134.0, 118.0,142.0, 141.0, 126.0, ..."
31,Height: 32 Width: 32,dog,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 9.260188102722168, ...","[127.0, 130.0, 81.0,130.0, 133.0, 88.0, ..."
33,Height: 32 Width: 32,dog,"[0.1307867169380188,0.7276672124862671, 0.0, ...","[118.0, 113.0, 81.0,122.0, 117.0, 83.0, ..."
39,Height: 32 Width: 32,dog,"[0.011272192001342773,0.0, 0.7608823180198669, ...","[72.0, 88.0, 97.0, 55.0,70.0, 84.0, 61.0, 73.0, ..."
42,Height: 32 Width: 32,dog,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 8.335664749145508, ...","[149.0, 149.0, 150.0,149.0, 149.0, 146.0, ..."
85,Height: 32 Width: 32,dog,"[0.0,0.29846465587615967, ...","[113.0, 174.0, 228.0,113.0, 174.0, 227.0, ..."
101,Height: 32 Width: 32,dog,"[0.5911874771118164, 0.0,0.0, 0.0, ...","[60.0, 77.0, 93.0, 60.0,76.0, 92.0, 58.0, 74.0, ..."
128,Height: 32 Width: 32,dog,"[0.0, 0.0,0.07171213626861572, ...","[86.0, 18.0, 39.0, 84.0,18.0, 38.0, 82.0, 16.0, ..."


In [23]:
dog_cat_neighbors = knn_cat.query(image_test_dog, k=1)
dog_automobile_neighbors = knn_automobile.query(image_test_dog, k=1)
dog_bird_neighbors = knn_bird.query(image_test_dog, k=1)
dog_dog_neighbors = knn_dog.query(image_test_dog, k=1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 63000   | 12.3772     | 285.477ms    |

| Done         | 509000  | 100         | 331.818ms    |

+--------------+---------+-------------+--------------+

In [ ]:
dog_cat_neighbors

In [ ]:
dog_dog_neighbors

In [26]:
dog_distances= turicreate.SFrame({'dog-dog': dog_dog_neighbors['distance'], 'dog-cat': dog_cat_neighbors['distance'],'dog-automobile': dog_automobile_neighbors['distance'], 'dog-bird': dog_bird_neighbors['distance']})

In [27]:
dog_distances

dog-automobile,dog-bird,dog-cat,dog-dog
41.95797614571203,41.75386473035126,36.419607706754384,33.47735903726335
46.00213318067788,41.3382958924861,38.83532688735542,32.84584956840554
42.946229069238804,38.615759085289056,36.97634108541546,35.03970731890584
41.68660600484793,37.08922699538214,34.575007291446106,33.90103276968193
39.22696649347584,38.27228869398105,34.77882479101661,37.484925090925636
40.58451176980721,39.146208923590486,35.11715782924591,34.94516534398124
45.10673529610854,40.523040105962316,40.60958309132649,39.095727834463545
41.32211409739762,38.19479183926956,39.90368673062214,37.76961310322034
41.82446549950164,40.156713166131446,38.067470016821176,35.10891446032838
45.497692940110376,45.55979626027668,42.72587329506032,43.242283258453455


In [28]:
dog_distances[0]['dog-cat']

36.419607706754384

In [29]:
def is_dog_correct(row):
    if dog_distances[row]['dog-dog'] < dog_distances[row]['dog-cat'] and dog_distances[row]['dog-dog']< dog_distances[row]['dog-automobile'] and dog_distances[row]['dog-dog'] < dog_distances[row]['dog-bird']: 
        return 1
    else:
        return 0

In [30]:
is_dog_correct(4)

0

In [ ]:
a=dog_distances(dog_distances.apply(lambda x: is_dog_correct(x)))
SFrame

In [ ]:
dog_distances.SFrame.apply(is_dog_correct)

In [ ]:
a= turicreate.SFrame({'is_dog_correct': dog_distances.apply(is_dog_correct(1))})

In [ ]:
sf.apply(lambda x: str(x['user_id']) + str(x['movie_id']) + str(x['rating']))


In [ ]:
dog_distances.apply(lambda x: is_dog_correct(x))

In [31]:
count=0
for i in range(len(dog_distances)):
    count = count + is_dog_correct(i);    
print(count)

678
